In [ ]:
import cv2
import os
import pandas as pd
import numpy as np

DATASET_DIR = 'dataset'
IMG_WIDTH, IMG_HEIGHT = 160, 120

df_balanced = pd.read_csv("dataset/labels_balanced.csv")

images = []
labels = []

for _, row in df_balanced.iterrows():
    frame_id = f"{int(row['frame_id']):06d}.png"
    img_path = os.path.join('images', frame_id)

    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        img = img / 255.0  # Normalizar [0,1]
        images.append(img)
        labels.append([row['steer'], row['throttle'], row['brake']])

X = np.array(images, dtype=np.float32)
y = np.array(labels, dtype=np.float32)

print(f"Datos cargados tras balanceo: {X.shape}, {y.shape}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Porcentajes de partición
TEST_SIZE = 0.1       # 10% test
VAL_SIZE = 0.15        # 15% validación (del total)

# Separar TEST
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=42
)

# Separar VALIDACIÓN del resto (proporcionalmente)
val_rel = VAL_SIZE / (1 - TEST_SIZE)  # fracción relativa de lo que queda
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_rel, random_state=42
)

print(f"Train: {len(X_train)} muestras")
print(f"Val:   {len(X_val)} muestras")
print(f"Test:  {len(X_test)} muestras")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,
                         weights='imagenet')

# Congelar las capas base (para no sobreentrenar)
base_model.trainable = False

# Añadir nuestro "cabezal" de regresión
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(3, activation='tanh')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='mse')
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 100, batch_size=32)



In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import numpy as np

def steer_to_class(steer):
    if steer < -0.35:
        return 0  # izquierda
    elif steer > 0.35:
        return 2  # derecha
    else:
        return 1  # recto


y_true_class = np.array([steer_to_class(s) for s in y_test[:, 0]])
y_pred_class = np.array([steer_to_class(s) for s in y_pred[:, 0]])

cm = confusion_matrix(y_true_class, y_pred_class, labels=[0,1,2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Izquierda","Recto","Derecha"])
disp.plot(cmap="Blues", values_format="d")
plt.title("Matriz de confusión para 'steer'")
plt.show()

In [ ]:
model.save('MobileNet_Dataset_balanced.h5')

In [ ]:
import matplotlib.pyplot as plt

# El historial de tu último entrenamiento está en la variable 'history'
history_dict = history.history

# Extraer los valores de pérdida (error)
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# Crear un rango de épocas para el eje X
epochs_range = range(1, len(loss) + 1)

# Graficar la pérdida de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(epochs_range, loss, 'b-o', label='Pérdida de Entrenamiento')
plt.plot(epochs_range, val_loss, 'r-o', label='Pérdida de Validación')
plt.title('Evolución de la Pérdida (Error) Durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (Loss)')
plt.legend()
plt.grid(True)
plt.show() 